In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sportsreference.fb.team import Team
from sportsreference.fb.schedule import Schedule

In [2]:
#Update these yearly to match Regulation
EPLClubs = [
    'Arsenal', 'Aston Villa', 'Brighton and Hove Albion', 'Burnley', 'Chelsea', 
    'Crystal Palace', 'Everton', 'Fulham', 'Leeds United', 'Leicester City', 
    'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United', 
    'Sheffield United', 'Southampton', 'Tottenham Hotspur', 'West Bromwich Albion', 
    'West Ham United', 'Wolverhampton Wanderers'
]

In [12]:
def build_dataframe(EPLClubs):
    df = pd.DataFrame(EPLClubs, columns = ['Name'])
    df['Record'] = Team('Arsenal').record
    df['Points'] = Team('Arsenal').points
    df['Goals Scored'] = Team('Arsenal').goals_scored
    df['Goals Given'] = Team('Arsenal').goals_against
    return df

In [14]:
df = pd.DataFrame(columns=('Name','Record','Points','Goals Scored', 'Goals Conceded'))
for i in range(EPLClubs):
    df.loc[i] = [i, Team(i).record, Team(i).points, Team(i).goals_scored, Team(i).goals_against]

,Name,Record,Points,Goals Scored,Goals Conceded


In [13]:
epl_df = build_dataframe(EPLClubs)
epl_df.head()

,Name,Record,Points,Goals Scored,Goals Given
0,Arsenal,7-2-3,23,40,11
1,Aston Villa,7-2-3,23,40,11
2,Brighton and Hove Albion,7-2-3,23,40,11
3,Burnley,7-2-3,23,40,11
4,Chelsea,7-2-3,23,40,11
